In [434]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
## 위는 기존 분석의 패키지, 아래는 새 분석의 패키지

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

In [435]:
import os
os.chdir(r"C:\Users\Sedaily\Desktop\개인\동국대 경영정보\2023년 2학기\머신러닝-딥러닝")
df=pd.read_csv('./shopping_trends_updated.csv')
df.head()

## 엑셀 데이터 불러오기

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [436]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])
df.head()

## 기존 분석, 문자 데이터의 숫자 변환

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,1,2,1,53,16,0,7,3,3.1,1,1,1,1,14,5,3
1,2,19,1,23,1,64,18,0,12,3,3.1,1,1,1,1,2,1,3
2,3,50,1,11,1,73,20,2,12,1,3.1,1,2,1,1,23,2,6
3,4,21,1,14,2,90,38,1,12,1,3.5,1,3,1,1,49,4,6
4,5,45,1,2,1,49,36,1,21,1,2.7,1,2,1,1,31,4,0


In [437]:
X = df[['Age','Gender','Location','Payment Method','Frequency of Purchases','Discount Applied']] 
y = df['Subscription Status']

##X, Y 변수 정의

In [438]:
df1 = df.drop(['Customer ID', 'Item Purchased', 'Category', 'Purchase Amount (USD)', 'Size', 'Color', 'Season', 'Review Rating', 'Shipping Type', 'Promo Code Used','Previous Purchases'], axis=1)
df2 = df1.reindex(['Age','Gender','Location','Payment Method','Discount Applied','Frequency of Purchases','Subscription Status'], axis=1)
df2

## X,Y 열(칼럼) 순서 정렬

,Age,Gender,Location,Payment Method,Discount Applied,Frequency of Purchases,Subscription Status
0,55,1,16,5,1,3,1
1,19,1,18,1,1,3,1
2,50,1,20,2,1,6,1
3,21,1,38,4,1,6,1
4,45,1,36,4,1,0,1
...,...,...,...,...,...,...,...
3895,40,0,45,5,0,6,0
3896,52,0,14,0,0,1,0
3897,46,0,29,5,0,5,0
3898,44,0,22,5,0,6,0


In [439]:
x_data = df2.iloc[:,:-1]
x_data

,Age,Gender,Location,Payment Method,Discount Applied,Frequency of Purchases
0,55,1,16,5,1,3
1,19,1,18,1,1,3
2,50,1,20,2,1,6
3,21,1,38,4,1,6
4,45,1,36,4,1,0
...,...,...,...,...,...,...
3895,40,0,45,5,0,6
3896,52,0,14,0,0,1
3897,46,0,29,5,0,5
3898,44,0,22,5,0,6


In [440]:
y_label = df2.iloc[:,-1]
y_label

0       1
1       1
2       1
3       1
4       1
       ..
3895    0
3896    0
3897    0
3898    0
3899    0
Name: Subscription Status, Length: 3900, dtype: int32

In [441]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label,test_size = 0.2, random_state=14)

## 훈련 데이터, 테스트 데이터 분할

In [442]:
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

## values로 판다스 데이터프레임을 넘파이로 변환

In [443]:
batch_size = 200

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

## 학습시 나눠진 데이터 세트 크기

In [444]:
model = nn.Sequential(
    nn.Linear(6, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

## 선형 신경망 모델, 입력 6개, 출력 2개 

In [445]:
batch_class, batch_labels = next(iter(train_loader))

## 배치 클래스, 라벨 정의

In [446]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 손실함수 크로스엔트로피로 정의

In [447]:
num_epochs = 2000

for epoch in range(num_epochs):
    
    total_batch = len(x_train) // batch_size
    
    for i, (batch_class, batch_labels) in enumerate(train_loader):
        
        X = batch_class.view(-1, 6)
        Y = batch_labels
        
        pred = model(X)
        Y = Y.long()
        cost = loss(pred, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
## 반복학습 수 증가할수록 정확도 개선되지만 80% 수준에서 한계

In [448]:
correct = 0
total = 0

for classes, labels in test_dataset:
    
    classes  = classes.view(-1, 6)
    outputs = model(classes)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += 1
    correct += (predicted == labels).sum()
    
print('Accuracy of test classes: %f %%' % (100 * float(correct) / total))

Accuracy of test classes: 82.948718 %
